In [1]:
from utils import *
from model2 import build_decoder_model

%load_ext autoreload
%autoreload 2

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [3]:
model = build_decoder_model()
model.load_model("no_emb2")
torch.cuda.memory_allocated()

271219712

In [4]:
class TestBatcher():
    
    def __init__(self, vocab, filename, max_article_len, max_target_len):
        generator = example_gen(filename)

        unknown_words_cnt = 0
        self.articles = []
        self.targets = []
        self.unknown_words = []
        self.batches = []
        while True:
            
            try:
                article_text, abstract_text = next(generator)
                
                article_ids, unknown_words_list = article2ids(article_text, vocab)
                art_len = min(max_article_len, len(article_ids))
                article_ids = article_ids[:art_len]
                
                target = abstract2ids(abstract_text, vocab, unknown_words_list)
                tar_len = min(max_target_len, len(target))
                target = target[:tar_len - 1]
                target.append(STOP_DECODING_ID)
                
                self.articles.append(np.array([article_ids]).T)
                self.targets.append(np.array(target))
                self.unknown_words.append(unknown_words_list)
                unknown_words_cnt = max(len(unknown_words_list), unknown_words_cnt)
                self.batches.append((np.array([article_ids]).T, np.array(target), unknown_words_list))
                #print(article_text)
            except:
                break
        print(len(self.batches))
        print(unknown_words_cnt)
    
    def generator(self):
        for batch in self.batches:
            yield batch

In [5]:
test_data = TestBatcher(vocab, "finished_files/test.bin", 400, 120)

11490
193


In [6]:
import codecs
import os

def test_generate(model, data, beam_width, mod=0):  
    model.train(False)
    
    with torch.autograd.set_grad_enabled(False):
        for i, (article_text, target, unk_words) in enumerate(data.generator()):
            result = generate_summary(article_text, model, beam_width)
            target = clean_sentence(target)
            result = clean_sentence(result)
            #bleu = sentence_bleu([result], target, weights=bleu_weights)
            result = outputids2words(result, vocab, unk_words)
            target = outputids2words(target, vocab, unk_words)
            #print('result is \n')
            #print(result.split())
            #print('target is \n')
            #print(target.split())
            if not os.access('refs' + str(mod), os.F_OK):
                os.mkdir('refs' + str(mod), 777)
            if not os.access('res' + str(mod), os.F_OK):
                os.mkdir('res' + str(mod), 777)
            with codecs.open('refs' + str(mod) + '/ref.A.' + str(i) +'.txt', 'w', encoding='utf8') as reference_file:
                sentences = target.split('.')
                for s in sentences:
                    reference_file.write(s.strip() + '.\n')
                
                
            with codecs.open('res' + str(mod) + '/res.' + str(i) + '.txt', 'w', encoding='utf8') as result_file:
                sentences = result.split('.')
                for s in sentences:
                    result_file.write(s.strip() + '.\n')

In [7]:
test_generate(model,test_data, beam_width=1)

In [8]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res0'
r.model_dir = 'E:/NIR/drag/refs0'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.A.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-05-10 13:28:31,283 [MainThread  ] [INFO ]  Writing summaries.
2019-05-10 13:28:31,284 [MainThread  ] [INFO ]  Processing summaries. Saving system files to C:\Users\Anatoly\AppData\Local\Temp\tmp4x9at37o\system and model files to C:\Users\Anatoly\AppData\Local\Temp\tmp4x9at37o\model.
2019-05-10 13:28:31,285 [MainThread  ] [INFO ]  Processing files in E:/NIR/drag/res0.
2019-05-10 13:28:31,290 [MainThread  ] [INFO ]  Processing res.0.txt.
2019-05-10 13:28:31,296 [MainThread  ] [INFO ]  Processing res.1.txt.
2019-05-10 13:28:31,301 [MainThread  ] [INFO ]  Processing res.10.txt.
2019-05-10 13:28:31,307 [MainThread  ] [INFO ]  Processing res.100.txt.
2019-05-10 13:28:31,315 [MainThread  ] [INFO ]  Processing res.1000.txt.
2019-05-10 13:28:31,320 [MainThread  ] [INFO ]  Processing res.10000.txt.
2019-05-10 13:28:31,326 [MainThread  ] [INFO ]  Processing res.10001.txt.
2019-05-10 13:28:31,332 [MainThread  ] [INFO ]  Processing res.10002.txt.
2019-05-10 13:28:31,337 [MainThread  ] [INFO ] 

2019-05-10 13:28:31,895 [MainThread  ] [INFO ]  Processing res.10091.txt.
2019-05-10 13:28:31,901 [MainThread  ] [INFO ]  Processing res.10092.txt.
2019-05-10 13:28:31,907 [MainThread  ] [INFO ]  Processing res.10093.txt.
2019-05-10 13:28:31,912 [MainThread  ] [INFO ]  Processing res.10094.txt.
2019-05-10 13:28:31,917 [MainThread  ] [INFO ]  Processing res.10095.txt.
2019-05-10 13:28:31,924 [MainThread  ] [INFO ]  Processing res.10096.txt.
2019-05-10 13:28:31,929 [MainThread  ] [INFO ]  Processing res.10097.txt.
2019-05-10 13:28:31,934 [MainThread  ] [INFO ]  Processing res.10098.txt.
2019-05-10 13:28:31,940 [MainThread  ] [INFO ]  Processing res.10099.txt.
2019-05-10 13:28:31,946 [MainThread  ] [INFO ]  Processing res.101.txt.
2019-05-10 13:28:31,951 [MainThread  ] [INFO ]  Processing res.1010.txt.
2019-05-10 13:28:31,957 [MainThread  ] [INFO ]  Processing res.10100.txt.
2019-05-10 13:28:31,962 [MainThread  ] [INFO ]  Processing res.10101.txt.
2019-05-10 13:28:31,967 [MainThread  ] [I

2019-05-10 13:28:32,517 [MainThread  ] [INFO ]  Processing res.10191.txt.
2019-05-10 13:28:32,523 [MainThread  ] [INFO ]  Processing res.10192.txt.
2019-05-10 13:28:32,529 [MainThread  ] [INFO ]  Processing res.10193.txt.
2019-05-10 13:28:32,534 [MainThread  ] [INFO ]  Processing res.10194.txt.
2019-05-10 13:28:32,539 [MainThread  ] [INFO ]  Processing res.10195.txt.
2019-05-10 13:28:32,545 [MainThread  ] [INFO ]  Processing res.10196.txt.
2019-05-10 13:28:32,550 [MainThread  ] [INFO ]  Processing res.10197.txt.
2019-05-10 13:28:32,557 [MainThread  ] [INFO ]  Processing res.10198.txt.
2019-05-10 13:28:32,562 [MainThread  ] [INFO ]  Processing res.10199.txt.
2019-05-10 13:28:32,567 [MainThread  ] [INFO ]  Processing res.102.txt.
2019-05-10 13:28:32,573 [MainThread  ] [INFO ]  Processing res.1020.txt.
2019-05-10 13:28:32,578 [MainThread  ] [INFO ]  Processing res.10200.txt.
2019-05-10 13:28:32,583 [MainThread  ] [INFO ]  Processing res.10201.txt.
2019-05-10 13:28:32,590 [MainThread  ] [I

2019-05-10 13:28:33,138 [MainThread  ] [INFO ]  Processing res.10291.txt.
2019-05-10 13:28:33,144 [MainThread  ] [INFO ]  Processing res.10292.txt.
2019-05-10 13:28:33,149 [MainThread  ] [INFO ]  Processing res.10293.txt.
2019-05-10 13:28:33,155 [MainThread  ] [INFO ]  Processing res.10294.txt.
2019-05-10 13:28:33,161 [MainThread  ] [INFO ]  Processing res.10295.txt.
2019-05-10 13:28:33,166 [MainThread  ] [INFO ]  Processing res.10296.txt.
2019-05-10 13:28:33,172 [MainThread  ] [INFO ]  Processing res.10297.txt.
2019-05-10 13:28:33,177 [MainThread  ] [INFO ]  Processing res.10298.txt.
2019-05-10 13:28:33,183 [MainThread  ] [INFO ]  Processing res.10299.txt.
2019-05-10 13:28:33,189 [MainThread  ] [INFO ]  Processing res.103.txt.
2019-05-10 13:28:33,194 [MainThread  ] [INFO ]  Processing res.1030.txt.
2019-05-10 13:28:33,199 [MainThread  ] [INFO ]  Processing res.10300.txt.
2019-05-10 13:28:33,205 [MainThread  ] [INFO ]  Processing res.10301.txt.
2019-05-10 13:28:33,211 [MainThread  ] [I

2019-05-10 13:28:33,756 [MainThread  ] [INFO ]  Processing res.10390.txt.
2019-05-10 13:28:33,763 [MainThread  ] [INFO ]  Processing res.10391.txt.
2019-05-10 13:28:33,767 [MainThread  ] [INFO ]  Processing res.10392.txt.
2019-05-10 13:28:33,772 [MainThread  ] [INFO ]  Processing res.10393.txt.
2019-05-10 13:28:33,778 [MainThread  ] [INFO ]  Processing res.10394.txt.
2019-05-10 13:28:33,783 [MainThread  ] [INFO ]  Processing res.10395.txt.
2019-05-10 13:28:33,789 [MainThread  ] [INFO ]  Processing res.10396.txt.
2019-05-10 13:28:33,794 [MainThread  ] [INFO ]  Processing res.10397.txt.
2019-05-10 13:28:33,799 [MainThread  ] [INFO ]  Processing res.10398.txt.
2019-05-10 13:28:33,806 [MainThread  ] [INFO ]  Processing res.10399.txt.
2019-05-10 13:28:33,811 [MainThread  ] [INFO ]  Processing res.104.txt.
2019-05-10 13:28:33,816 [MainThread  ] [INFO ]  Processing res.1040.txt.
2019-05-10 13:28:33,822 [MainThread  ] [INFO ]  Processing res.10400.txt.
2019-05-10 13:28:33,828 [MainThread  ] [I

...

2019-05-10 13:30:43,476 [MainThread  ] [INFO ]  Processing ref.A.969.txt.
2019-05-10 13:30:43,482 [MainThread  ] [INFO ]  Processing ref.A.9690.txt.
2019-05-10 13:30:43,487 [MainThread  ] [INFO ]  Processing ref.A.9691.txt.
2019-05-10 13:30:43,493 [MainThread  ] [INFO ]  Processing ref.A.9692.txt.
2019-05-10 13:30:43,499 [MainThread  ] [INFO ]  Processing ref.A.9693.txt.
2019-05-10 13:30:43,505 [MainThread  ] [INFO ]  Processing ref.A.9694.txt.
2019-05-10 13:30:43,512 [MainThread  ] [INFO ]  Processing ref.A.9695.txt.
2019-05-10 13:30:43,517 [MainThread  ] [INFO ]  Processing ref.A.9696.txt.
2019-05-10 13:30:43,522 [MainThread  ] [INFO ]  Processing ref.A.9697.txt.
2019-05-10 13:30:43,529 [MainThread  ] [INFO ]  Processing ref.A.9698.txt.
2019-05-10 13:30:43,534 [MainThread  ] [INFO ]  Processing ref.A.9699.txt.
2019-05-10 13:30:43,539 [MainThread  ] [INFO ]  Processing ref.A.97.txt.
2019-05-10 13:30:43,545 [MainThread  ] [INFO ]  Processing ref.A.970.txt.
2019-05-10 13:30:43,551 [Main

2019-05-10 13:30:44,120 [MainThread  ] [INFO ]  Processing ref.A.9789.txt.
2019-05-10 13:30:44,128 [MainThread  ] [INFO ]  Processing ref.A.979.txt.
2019-05-10 13:30:44,134 [MainThread  ] [INFO ]  Processing ref.A.9790.txt.
2019-05-10 13:30:44,139 [MainThread  ] [INFO ]  Processing ref.A.9791.txt.
2019-05-10 13:30:44,145 [MainThread  ] [INFO ]  Processing ref.A.9792.txt.
2019-05-10 13:30:44,150 [MainThread  ] [INFO ]  Processing ref.A.9793.txt.
2019-05-10 13:30:44,156 [MainThread  ] [INFO ]  Processing ref.A.9794.txt.
2019-05-10 13:30:44,162 [MainThread  ] [INFO ]  Processing ref.A.9795.txt.
2019-05-10 13:30:44,168 [MainThread  ] [INFO ]  Processing ref.A.9796.txt.
2019-05-10 13:30:44,173 [MainThread  ] [INFO ]  Processing ref.A.9797.txt.
2019-05-10 13:30:44,179 [MainThread  ] [INFO ]  Processing ref.A.9798.txt.
2019-05-10 13:30:44,184 [MainThread  ] [INFO ]  Processing ref.A.9799.txt.
2019-05-10 13:30:44,190 [MainThread  ] [INFO ]  Processing ref.A.98.txt.
2019-05-10 13:30:44,195 [Mai

2019-05-10 13:30:44,801 [MainThread  ] [INFO ]  Processing ref.A.9888.txt.
2019-05-10 13:30:44,807 [MainThread  ] [INFO ]  Processing ref.A.9889.txt.
2019-05-10 13:30:44,815 [MainThread  ] [INFO ]  Processing ref.A.989.txt.
2019-05-10 13:30:44,820 [MainThread  ] [INFO ]  Processing ref.A.9890.txt.
2019-05-10 13:30:44,822 [MainThread  ] [INFO ]  Processing ref.A.9891.txt.
2019-05-10 13:30:44,829 [MainThread  ] [INFO ]  Processing ref.A.9892.txt.
2019-05-10 13:30:44,834 [MainThread  ] [INFO ]  Processing ref.A.9893.txt.
2019-05-10 13:30:44,839 [MainThread  ] [INFO ]  Processing ref.A.9894.txt.
2019-05-10 13:30:44,846 [MainThread  ] [INFO ]  Processing ref.A.9895.txt.
2019-05-10 13:30:44,851 [MainThread  ] [INFO ]  Processing ref.A.9896.txt.
2019-05-10 13:30:44,858 [MainThread  ] [INFO ]  Processing ref.A.9897.txt.
2019-05-10 13:30:44,864 [MainThread  ] [INFO ]  Processing ref.A.9898.txt.
2019-05-10 13:30:44,869 [MainThread  ] [INFO ]  Processing ref.A.9899.txt.
2019-05-10 13:30:44,875 [M

2019-05-10 13:30:45,451 [MainThread  ] [INFO ]  Processing ref.A.9987.txt.
2019-05-10 13:30:45,457 [MainThread  ] [INFO ]  Processing ref.A.9988.txt.
2019-05-10 13:30:45,470 [MainThread  ] [INFO ]  Processing ref.A.9989.txt.
2019-05-10 13:30:45,475 [MainThread  ] [INFO ]  Processing ref.A.999.txt.
2019-05-10 13:30:45,481 [MainThread  ] [INFO ]  Processing ref.A.9990.txt.
2019-05-10 13:30:45,487 [MainThread  ] [INFO ]  Processing ref.A.9991.txt.
2019-05-10 13:30:45,493 [MainThread  ] [INFO ]  Processing ref.A.9992.txt.
2019-05-10 13:30:45,499 [MainThread  ] [INFO ]  Processing ref.A.9993.txt.
2019-05-10 13:30:45,504 [MainThread  ] [INFO ]  Processing ref.A.9994.txt.
2019-05-10 13:30:45,511 [MainThread  ] [INFO ]  Processing ref.A.9995.txt.
2019-05-10 13:30:45,516 [MainThread  ] [INFO ]  Processing ref.A.9996.txt.
2019-05-10 13:30:45,521 [MainThread  ] [INFO ]  Processing ref.A.9997.txt.
2019-05-10 13:30:45,528 [MainThread  ] [INFO ]  Processing ref.A.9998.txt.
2019-05-10 13:30:45,533 [M

---------------------------------------------
1 ROUGE-1 Average_R: 0.09754 (95%-conf.int. 0.09665 - 0.09855)
1 ROUGE-1 Average_P: 0.56358 (95%-conf.int. 0.55903 - 0.56808)
1 ROUGE-1 Average_F: 0.16318 (95%-conf.int. 0.16179 - 0.16478)
---------------------------------------------
1 ROUGE-2 Average_R: 0.04033 (95%-conf.int. 0.03958 - 0.04107)
1 ROUGE-2 Average_P: 0.26510 (95%-conf.int. 0.26039 - 0.27003)
1 ROUGE-2 Average_F: 0.06867 (95%-conf.int. 0.06739 - 0.06994)
---------------------------------------------
1 ROUGE-3 Average_R: 0.01855 (95%-conf.int. 0.01791 - 0.01917)
1 ROUGE-3 Average_P: 0.13944 (95%-conf.int. 0.13508 - 0.14416)
1 ROUGE-3 Average_F: 0.03208 (95%-conf.int. 0.03100 - 0.03313)
---------------------------------------------
1 ROUGE-4 Average_R: 0.01008 (95%-conf.int. 0.00957 - 0.01058)
1 ROUGE-4 Average_P: 0.08761 (95%-conf.int. 0.08363 - 0.09180)
1 ROUGE-4 Average_F: 0.01770 (95%-conf.int. 0.01682 - 0.01855)
---------------------------------------------
1 ROUGE-L Aver

model.load_model("cov_loss2")

test_generate(model,test_data, beam_width=4, mod=1)

r = Rouge155()
r.system_dir = 'E:/NIR/drag/res1'
r.model_dir = 'E:/NIR/drag/refs1'
r.system_filename_pattern = 'res.(\d+).txt'
r.model_filename_pattern = 'ref.A.#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)